In [2]:
import cvxpy as cp 
import numpy as np

In [15]:

a = np.empty((5,6))
b = np.empty((6,100,4))

(a @ b).shape

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 100 is different from 6)

In [33]:
eigensum_expr = "efbc,diaf,ghde->ghiabc"

gates = [np.random.rand(*shape) for shape in [(4, 512, 4, 512), (2, 512, 2, 512), (2, 4, 2, 4)]]

reconstructed_tensor = np.einsum(eigensum_expr, *gates, optimize=True)

In [61]:
def hardcoded_3_qubit_reconstruct(gates):
    # gate_1 of shape i1,,i2,k1,k2
    # gate_2 of shape k1,i3,j1,k3
    # gate_3 of shape k2,k3,j2,j3

    gate_3, gate_2, gate_1 = gates
    i1, i2, k1, k2 = gate_1.shape
    # print("i1=",i1,"i2=",i2,"k1=",k1,"k2=",k2)
    k1, i3, j1, k3 = gate_2.shape
    # print("k1=",k1,"i3=",i3,"j1=",j1,"k3=",k3)
    k2, k3, j2, j3 = gate_3.shape
    print("i1=",i1,"i2=",i2, "i3=",i3)
    print("j1=",j1,"j2=",j2,"j3=",j3)
    print("k1=",k1,"k2=",k2,"k3=",k3)

    #multiply gate_2 and gate_3 along k_3
    print("gate3.shape",gate_3.shape)
    print(np.transpose(gate_3,(1,0,2,3)).shape)
    gate_23 = gate_2.reshape(k1 * i3 * j1, k3) @ np.transpose(gate_3,(1,0,2,3)).reshape((k3, k2 * j2 * j3))
    #shape k1*i3*j1, k2*j2*j3
    print(gate_23.shape)
    print(gate_23.reshape((k1, i3, j1, k2, j2, j3)).shape)
    gate_23 = np.transpose(gate_23.reshape((k1, i3, j1, k2, j2, j3)),(0,3,1,2,4,5)) #shape (k1, k2, i3, j1, j2, j3)
    print(gate_23.shape)
    gate_23 = gate_23.reshape((k1*k2, i3*j1*j2 * j3))
    #multiply gate_1 and gate_23 along k_1*k_2
    gate_123 = gate_1.reshape((i1*i2, k1*k2)) @ gate_23
    gate_123 = gate_123.reshape((i1, i2, i3, j1, j2, j3))
    return gate_123

In [67]:
def hardcoded_3_qubit_reconstruct(gates):
    # gate_1 of shape i1,,i2,k1,k2
    # gate_2 of shape k1,i3,j1,k3
    # gate_3 of shape k2,k3,j2,j3

    gate_3, gate_2, gate_1 = gates
    i1, i2, k1, k2 = gate_1.shape
    k1, i3, j1, k3 = gate_2.shape
    k2, k3, j2, j3 = gate_3.shape

    #multiply gate_2 and gate_3 along k_3
    gate_23 = gate_2.reshape(k1 * i3 * j1, k3) @ cp.transpose(gate_3,(1,0,2,3)).reshape((k3, k2 * j2 * j3), order='C')
    #shape k1*i3*j1, k2*j2*j3
    gate_23 = cp.transpose(gate_23.reshape((k1, i3, j1, k2, j2, j3),order = 'C'),(0,3,1,2,4,5)) #shape (k1, k2, i3, j1, j2, j3)
    gate_23 = gate_23.reshape((k1*k2, i3*j1*j2 * j3), order = 'C')
    #multiply gate_1 and gate_23 along k_1*k_2
    gate_123 = gate_1.reshape((i1*i2, k1*k2), order = 'C') @ gate_23
    gate_123 = gate_123.reshape((i1, i2, i3, j1, j2, j3), order = 'C')
    return gate_123

reconstructed_tensor2 = hardcoded_3_qubit_reconstruct(gates)

In [69]:
np.allclose(reconstructed_tensor2.value, reconstructed_tensor)

True

In [ ]:
gate_3, gate_2, gate_1 = gates
i1, i2, k1, k2 = gate_1.shape
k1, i3, j1, k3 = gate_2.shape
k2, k3, j2, j3 = gate_3.shape

gate_3 = cp.Variable(gate_3.shape)
gate_2 = cp.Variable(gate_2.shape)
gate_1 = cp.Variable(gate_1.shape)

gate_1 = gate_1.reshape(i1*i2, k1*k2)


gate_23 = gate_2.reshape(k1*i3*i2,k3) @ gate_3.reshape

In [28]:
a = cp.Variable((1,2,3,4))
print(a.shape)
a[0] = a.transpose(0,2,1,3)
print(a.shape)
print(a.T.shape)
# print(a.reshape((k2,k3, j2*j3), order = 'C').shape)
# a.reshape((k2,k3, j2*j3), order = 'C').T

(1, 2, 3, 4)


AttributeError: 'Variable' object has no attribute 'transpose'

In [ ]:
cp.transpose(np.empty((1,2,3,4)), (0,2,1,3))

(4, 3, 2, 1)